In [1]:
import os
import pandas as pd
import math 
from collections import defaultdict 

DATA_DIR="../data"

In [2]:
# Get the list of files in the directory
files = os.listdir(DATA_DIR)

# Print the list of files
for file in files:
    print(file)

super_families_indices.pickle
CD_models_curated_sorted
SPARCLE_IDS_curated.mtx4ml.tsv.gz
curated_names_unique
CD_models_curated
super_families_curated
SPARCLE_IDS_curated.csv
CD_models_indices.pickle
SPARCLE_IDS_curated_simplified.csv
super_families_curated_sorted


## Import preprocessed data

### Import input

In [ ]:
# Access &read the file 
file_path = os.path.join(DATA_DIR, files[2])
df_one_hot = pd.read_csv(file_path, sep='\t', compression='gzip')

#test succ. loading
print(df_one_hot.head())

### Import labels

Currently not used

In [39]:
# Access &read the file 
print(files[8])
file_path = os.path.join(DATA_DIR, files[8])
df_labels_simplified = pd.read_csv(file_path, on_bad_lines  = "warn")

SPARCLE_IDS_curated_simplified.csv


In [38]:
df_labels_simplified

,ArchId,CurVer,CurLabel,CurName,CurName_simplified,superfamilyarch,SpecificArch,TitleStrings,Taxid,IsSpec,Status,LabelState,archLen_max,archLen_med,ArchId_string,SuperFamID_string
0,11436966,2,"(2,3-dihydroxybenzoyl)adenylate synthase catal...","(2,3-dihydroxybenzoyl)adenylate synthase","(2,3-dihydroxybenzoyl)adenylate synthase",AMP-binding,COG1021,Non-ribosomal peptide synthetase component E (...,1,1,published,curated,NaN,NaN,6446,0
1,11494111,2,"(2,3-dihydroxybenzoyl)adenylate synthase catal...","(2,3-dihydroxybenzoyl)adenylate synthase","(2,3-dihydroxybenzoyl)adenylate synthase",AMP-binding,TIGR02275,"2,3-dihydroxybenzoate-AMP ligase",1,1,published,curated,NaN,NaN,53958,0
2,11494111,2,"(2,3-dihydroxybenzoyl)adenylate synthase catal...","(2,3-dihydroxybenzoyl)adenylate synthase","(2,3-dihydroxybenzoyl)adenylate synthase",DHB_AMP_lig,TIGR02275,"2,3-dihydroxybenzoate-AMP ligase",1,1,published,curated,NaN,NaN,53958,37086
3,11436966,2,"(2,3-dihydroxybenzoyl)adenylate synthase catal...","(2,3-dihydroxybenzoyl)adenylate synthase","(2,3-dihydroxybenzoyl)adenylate synthase",EntE,COG1021,Non-ribosomal peptide synthetase component E (...,1,1,published,curated,NaN,NaN,6446,34064
4,11485118,2,"(2,3-dihydroxybenzoyl)adenylate synthase catal...","(2,3-dihydroxybenzoyl)adenylate synthase","(2,3-dihydroxybenzoyl)adenylate synthase",entE,PRK10946,"(2,3-dihydroxybenzoyl)adenylate synthase",1,1,published,curated,NaN,NaN,32482,35977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42761,11510,1,protein contains domain Zwilch,Zwilch domain-containing protein,Zwilch domain-containing protein,Zwilch,NaN,NaN,1,0,published,curated,NaN,NaN,0,10789
42762,10115543,2,ZZ-type zinc finger protein may function as a ...,ZZ-type zinc finger protein,ZZ-type zinc finger protein,ZZ,cd02249,NaN,1,1,published,curated,NaN,NaN,20057,295
42763,10115811,2,ZZ-type zinc finger protein may function as a ...,ZZ-type zinc finger protein,ZZ-type zinc finger protein,ZZ,cd02340,NaN,1,1,published,curated,NaN,NaN,20148,295
42764,10645330,2,ZZ-type zinc finger protein may function as a ...,ZZ-type zinc finger protein,ZZ-type zinc finger protein,ZZ,smart00291,"Zinc-binding domain, present in Dystrophin, CR...",1,1,published,curated,NaN,NaN,402,295


In [4]:
df_selCol = df_labels_simplified[['CurName_simplified', 'superfamilyarch', 'SpecificArch']]
len(df_selCol)

42766

In [5]:
# Remove rows where either 'SpecificArch' or 'CurName' is empty
df_selCol = df_selCol.dropna(subset=['SpecificArch', 'superfamilyarch', 'CurName_simplified'])
df_selCol = df_selCol[(df_selCol['SpecificArch'] != '') & (df_selCol['superfamilyarch'] != '') & (df_selCol['CurName_simplified'] != '')]
df_selCol.shape

(39584, 3)

## Split data set into training and testing data

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
#just for test and short running of problem: take first 1000 lines from df
df_final = df_one_hot#.head(5000)

In [6]:
X = df_final.drop(['CurName'], axis=1)  # Features
y = df_final['CurName']  # Target variable

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (34212, 48296)
Testing set size: (8554, 48296)


In [8]:
#test access
X_train.iloc[[29]]
y_train.iloc[[29]]

9339    ketoacyl-ACP synthase III
Name: CurName, dtype: object

## Decision Tree Classifier

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
# Initialize the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)

In [11]:
# Fit the model on the training data
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [12]:
# Make predictions and evaluate the model
predictions = clf.predict(X_test)

In [13]:
# Evaluate the model (e.g.: accuracy)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)

Accuracy: 0.39992985737666586


Further ideas: 
- 'superfamilyarch' seems to be more general, prioritize decision tree formation by first considering 'superfamilyarch' and then 'SpecificArch'?
- Parameter change: 'min_samples_split': determines the minimum number of samples required to split an internal node, increasing this number can help prevent overfitting

## Support Vector Machine (SVM)

In [14]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd

In [ ]:
svm_clf_poly = SVC(kernel='poly', random_state=42)
mod_svm_poly = svm_clf_poly.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
predictions = mod_svm_poly.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

In [24]:
# Initialize the SVM classifier
svm_clf = SVC(random_state=42)

In [ ]:
# Fit the model on the training data
svm_clf.fit(X_train, y_train)

In [71]:
# Make predictions on the test set
predictions = svm_clf.predict(X_test)

In [72]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

0.032

In [ ]:
# Define the parameter grid for SVC
param_grid = {
    'C': [0.1, 1, 10],  # Example parameter: Regularization strength
    'kernel': ['linear', 'rbf'],  # Example parameter: Type of SVM kernel
    # Add other parameters here if needed
}

In [ ]:
# Set n_jobs=-1 to use all available CPUs for parallelizing the search
grid_search = GridSearchCV(SVC(random_state=42), param_grid, n_jobs=-1, verbose=1)


In [ ]:
grid_search.fit(X_train, y_train_encoded)

In [ ]:
best_svc = grid_search.best_estimator_

In [ ]:
predictions = best_svc.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test_encoded, predictions)

In [ ]:
print("Best SVC Parameters:", grid_search.best_params_)
print("SVM Accuracy with Best Parameters:", accuracy)

## Principal Component Analysis

In [73]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [74]:
# standardize features before applying PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [78]:
# Initialize PCA - for example, to reduce the dataset to 2 principal components
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
# The number of components PCA chose to keep
print("Number of components chosen:", pca.n_components_)

Remember values above created have been only done with a subset of 5000 rows so not representative!

In [ ]:
#now re-run PCA or SVM with reduced dimensions!

In [80]:
%%bash 
pip freeze 

anyio==4.3.0
appnope==0.1.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
async-lru==2.0.4
attrs==23.2.0
Babel==2.14.0
beautifulsoup4==4.12.3
bleach==6.1.0
certifi==2024.2.2
cffi==1.16.0
charset-normalizer==3.3.2
comm==0.2.1
debugpy==1.8.1
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.2.0
executing==2.0.1
fastjsonschema==2.19.1
fqdn==1.5.1
h11==0.14.0
httpcore==1.0.4
httpx==0.27.0
idna==3.6
importlib-metadata==7.0.1
ipykernel==6.29.3
ipython==8.18.1
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.3
joblib==1.3.2
json5==0.9.17
jsonpointer==2.4
jsonschema==4.21.1
jsonschema-specifications==2023.12.1
jupyter-events==0.9.0
jupyter-lsp==2.2.3
jupyter_client==8.6.0
jupyter_core==5.7.1
jupyter_server==2.12.5
jupyter_server_terminals==0.5.2
jupyterlab==4.1.2
jupyterlab_pygments==0.3.0
jupyterlab_server==2.25.3
MarkupSafe==2.1.5
matplotlib-inline==0.1.6
mistune==3.0.2
nbclient==0.9.0
nbconvert==7.16.1
nbformat==5.9.2
nest-asyncio==1.6.0
notebook==7.1.1
note